In [1]:
import pandas as pd
import re
from IPython.display import display

# ===== 0) 파일 경로 =====
input_file  = "/Users/mac/Documents/SORA_Project/data/raw/Y14_경찰청_전국 치안센터 주소 현황_20250630_TM.csv"
output_file = "Y14_경찰청_전국 치안센터 주소 현황_20250630_TM_전처리.csv"


# ===== 1) 데이터 로드 =====
df = pd.read_csv(input_file, sep="\t", encoding="utf-8-sig")
print(f"✅ 원본 데이터 로드 완료: {df.shape}")
print("\n📌 데이터 기본 정보:")
df.info()


✅ 원본 데이터 로드 완료: (741, 18)

📌 데이터 기본 정보:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 741 entries, 0 to 740
Data columns (total 18 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   연번      741 non-null    int64  
 1   시도청     741 non-null    object 
 2   경찰서     741 non-null    object 
 3   치안센터명   741 non-null    object 
 4   주소      741 non-null    object 
 5   입력주소    741 non-null    object 
 6   X       741 non-null    int64  
 7   Y       741 non-null    int64  
 8   CLSS    741 non-null    object 
 9   PNU     711 non-null    float64
 10  주소구분    741 non-null    object 
 11  표준신주소   680 non-null    object 
 12  표준구주소   715 non-null    object 
 13  우편번호    666 non-null    float64
 14  행정동코드   688 non-null    float64
 15  행정동명    688 non-null    object 
 16  법정동코드   687 non-null    float64
 17  법정동명    687 non-null    object 
dtypes: float64(4), int64(3), object(11)
memory usage: 104.3+ KB


In [2]:
# ===== 2) 좌표 컬럼만 숫자형 변환 =====
coord_cols = ["위도", "경도", "X좌표", "Y좌표"]
for col in coord_cols:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors="coerce")

def format_coords(x, decimals):
    if pd.isna(x):
        return None
    try:
        return round(float(x), decimals)
    except:
        return x

if "위도" in df.columns:
    df["위도"] = df["위도"].apply(lambda x: format_coords(x, 6))
if "경도" in df.columns:
    df["경도"] = df["경도"].apply(lambda x: format_coords(x, 6))
if "X좌표" in df.columns:
    df["X좌표"] = df["X좌표"].apply(lambda x: format_coords(x, 4))
if "Y좌표" in df.columns:
    df["Y좌표"] = df["Y좌표"].apply(lambda x: format_coords(x, 5))

# ===== 3) 코드형 컬럼 문자열 정리 =====
code_cols = ["PNU", "행정동코드", "법정동코드", "시도코드", "시군구코드", "읍면동코드", "우편번호"]
def clean_code_str(x):
    if pd.isna(x):
        return None
    s = str(x).strip()
    s = re.sub(r"[^\d]", "", s)  # 숫자만 남기기 (소수점, 공백 제거)
    return s if s else None

for c in code_cols:
    if c in df.columns:
        df[c] = df[c].apply(clean_code_str)

# ===== 4) 주소 파싱 함수 =====
def parse_address(addr):
    if pd.isna(addr) or str(addr).strip() == "":
        return None, None, None, None
    parts = re.split(r"\s+", str(addr).strip())
    si   = parts[0] if len(parts) > 0 else None
    gu   = parts[1] if len(parts) > 1 and re.search(r"(시|군|구)$", parts[1]) else None
    dong = parts[2] if len(parts) > 2 else None
    road = " ".join(parts[3:]) if len(parts) > 3 else None
    return si, gu, dong, road

# 표준신주소 → 표준구주소 순으로 기준주소 선택
def pick_standard_address(row):
    if "표준신주소" in row and pd.notna(row["표준신주소"]) and str(row["표준신주소"]).strip() != "":
        return row["표준신주소"]
    if "표준구주소" in row and pd.notna(row["표준구주소"]) and str(row["표준구주소"]).strip() != "":
        return row["표준구주소"]
    return None

df["기준주소"] = df.apply(pick_standard_address, axis=1)
df[["시", "구", "동", "도로명"]] = df["기준주소"].apply(lambda x: pd.Series(parse_address(x)))

# ===== 5) NaN 보완 (도로명주소·지번주소 활용) =====
def fill_address_from_row(row):
    # 도로명주소 기준 보완
    if pd.notna(row.get("도로명주소")):
        parts = str(row["도로명주소"]).strip().split()
        if len(parts) >= 3:
            row["시"] = row["시"] or parts[0]
            row["구"] = row["구"] or parts[1]
            row["동"] = row["동"] or parts[2]
            if pd.isna(row["도로명"]) and len(parts) > 3:
                row["도로명"] = " ".join(parts[3:])
    # 지번주소 기준 보완
    if pd.notna(row.get("지번주소")):
        parts = str(row["지번주소"]).strip().split()
        if len(parts) >= 3:
            row["시"] = row["시"] or parts[0]
            row["구"] = row["구"] or parts[1]
            row["동"] = row["동"] or parts[2]
            if pd.isna(row["도로명"]) and len(parts) > 3:
                row["도로명"] = " ".join(parts[3:])
    return row

df = df.apply(fill_address_from_row, axis=1)

# ===== 6) 좌표 기반 보완 (있을 때만) =====
coord_lookup_cols = []
if "위도" in df.columns and "경도" in df.columns:
    coord_lookup_cols = ["위도", "경도"]
elif "X좌표" in df.columns and "Y좌표" in df.columns:
    coord_lookup_cols = ["X좌표", "Y좌표"]

if coord_lookup_cols:
    coord_lookup = df.dropna(subset=["시", "구", "동", "도로명"]).set_index(coord_lookup_cols)[["시", "구", "동", "도로명"]]

    def fill_from_coords(row):
        if any(pd.isna(row[c]) for c in ["시", "구", "동", "도로명"]) and tuple(row[coord_lookup_cols]) in coord_lookup.index:
            vals = coord_lookup.loc[tuple(row[coord_lookup_cols])]
            for c in ["시", "구", "동", "도로명"]:
                if pd.isna(row[c]):
                    row[c] = vals[c]
        return row

    df = df.apply(fill_from_coords, axis=1)
else:
    print("⚠️ 좌표 컬럼이 없어 좌표 기반 보완은 건너뜀")

# ===== 7) 컬럼 순서 정렬 =====
target_cols = ["시", "구", "동", "도로명"]
cols = list(df.columns)
if "유형명" in cols:
    type_idx = cols.index("유형명")
else:
    type_idx = 0

cols = [c for c in cols if c not in target_cols]
new_cols = cols[:type_idx+1] + target_cols + cols[type_idx+1:]
df = df[new_cols]

print("\n✅ 시/구/동/도로명 컬럼 이동 완료")
print(df.columns.tolist())

# ===== 8) 저장 =====
df.to_csv(output_file, index=False, encoding="utf-8-sig")
print(f"\n💾 전처리 완료 CSV 저장: {output_file}")

# ===== 9) 결과 확인 =====
print("\n🧾 결과 컬럼:")
print(df.columns.tolist())
display(df.head(10))

⚠️ 좌표 컬럼이 없어 좌표 기반 보완은 건너뜀

✅ 시/구/동/도로명 컬럼 이동 완료
['연번', '시', '구', '동', '도로명', '시도청', '경찰서', '치안센터명', '주소', '입력주소', 'X', 'Y', 'CLSS', 'PNU', '주소구분', '표준신주소', '표준구주소', '우편번호', '행정동코드', '행정동명', '법정동코드', '법정동명', '기준주소']

💾 전처리 완료 CSV 저장: Y14_경찰청_전국 치안센터 주소 현황_20250630_TM_전처리.csv

🧾 결과 컬럼:
['연번', '시', '구', '동', '도로명', '시도청', '경찰서', '치안센터명', '주소', '입력주소', 'X', 'Y', 'CLSS', 'PNU', '주소구분', '표준신주소', '표준구주소', '우편번호', '행정동코드', '행정동명', '법정동코드', '법정동명', '기준주소']


,연번,시,구,동,도로명,시도청,경찰서,치안센터명,주소,입력주소,...,PNU,주소구분,표준신주소,표준구주소,우편번호,행정동코드,행정동명,법정동코드,법정동명,기준주소
0,1,서울특별시,중구,충무로2길,39,서울청,서울중부,충4치안센터,서울 중구 충무로2길 39,서울 중구 충무로2길 39,...,11140132001002718,새주소,서울특별시 중구 충무로2길 39,서울특별시 중구 충무로4가 27-5,45560,11140590000,광희동,11140132000,충무로4가,서울특별시 중구 충무로2길 39
1,2,서울특별시,종로구,비봉길,13,서울청,서울종로,구기치안센터,서울 종로구 비봉길 13,서울 종로구 비봉길 13,...,1111018200101218,새주소,서울특별시 종로구 비봉길 13,서울특별시 종로구 구기동 120-16,30000,11110560000,평창동,11110182000,구기동,서울특별시 종로구 비봉길 13
2,3,서울특별시,서대문구,북아현로,59-1,서울청,서울서대문,북아현치안센터,서울 서대문구 북아현로 59-1번지,서울 서대문구 북아현로 59-1번지,...,11410110001017618,새주소,서울특별시 서대문구 북아현로 59-1,서울특별시 서대문구 북아현동 176-53,37620,11410565000,충현동,11410110000,북아현동,서울특별시 서대문구 북아현로 59-1
3,4,서울특별시,종로구,돈화문로,28,서울청,서울혜화,종로3가치안센터,서울 종로구 돈화문로 28,서울 종로구 돈화문로 28,...,11110151001005918,새주소,서울특별시 종로구 돈화문로 28,서울특별시 종로구 묘동 59-5,31380,11110615000,종로1.2.3.4가동,11110151000,묘동,서울특별시 종로구 돈화문로 28
4,5,서울특별시,용산구,청파로,274-2,서울청,서울용산,청파치안센터,서울 용산구 청파로 274-2,서울 용산구 청파로 274-2,...,1117011100100118,새주소,서울특별시 용산구 청파로 274-2,서울특별시 용산구 청파동3가 10-5,43130,11170555000,청파동,11170111000,청파동3가,서울특별시 용산구 청파로 274-2
5,6,서울특별시,용산구,신흥로,90,서울청,서울용산,용산치안센터,서울 용산구 신흥로 90,서울 용산구 신흥로 90,...,11170102001001118,새주소,서울특별시 용산구 신흥로 90,서울특별시 용산구 용산동2가 11-13,43390,11170520000,용산2가동,11170102000,용산동2가,서울특별시 용산구 신흥로 90
6,7,서울특별시,용산구,이태원로,281,서울청,서울용산,한남2치안센터,서울 용산구 이태원로 281,서울 용산구 이태원로 281,...,1117013100200100118,새주소,서울특별시 용산구 이태원로 281,서울특별시 용산구 한남동 산 10-33,43470,11170685000,한남동,11170131000,한남동,서울특별시 용산구 이태원로 281
7,8,서울특별시,용산구,이촌로71길,24,서울청,서울용산,이촌치안센터,서울 용산구 이촌로71길 24,서울 용산구 이촌로71길 24,...,11170129001030118,새주소,서울특별시 용산구 이촌로71길 24,서울특별시 용산구 이촌동 301-82,44220,11170630000,이촌1동,11170129000,이촌동,서울특별시 용산구 이촌로71길 24
8,9,서울특별시,성북구,보문로29길,92,서울청,서울성북,삼선치안센터,서울 성북구 보문로29길 92,서울 성북구 보문로29길 92,...,1129011300100680118,새주소,서울특별시 성북구 보문로29길 92,서울특별시 성북구 삼선동3가 68-6,28630,11290555000,삼선동,11290113000,삼선동3가,서울특별시 성북구 보문로29길 92
9,10,서울특별시,성북구,정릉로31길,30,서울청,서울성북,길음1치안센터,서울 성북구 정릉로 31길 30,서울 성북구 정릉로 31길 30,...,1129013300109660218,새주소,서울특별시 성북구 정릉로31길 30,서울특별시 성북구 정릉동 966-173,27190,11290620000,정릉1동,11290133000,정릉동,서울특별시 성북구 정릉로31길 30
